<a href="https://colab.research.google.com/github/solong96/solong96/blob/main/0818_cc_text_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.rnn import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout, Embedding
from keras.layers.normalization import batch_normalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
import os

In [ ]:
path = '/content/drive/MyDrive/0809/'
os.chdir(path)

In [ ]:
index_list = list(range(0,10,1))
word = pd.DataFrame({'cate':["energy","industry","waste","traffic","building","land-use","green","biodiversity","water","heat island"],
                                    'toxic':[11,12,13,14,15,16,21,22,23,24],
                                    'class':index_list})
word

,cate,toxic,class
0,energy,11,0
1,industry,12,1
2,waste,13,2
3,traffic,14,3
4,building,15,4
5,land-use,16,5
6,green,21,6
7,biodiversity,22,7
8,water,23,8
9,heat island,24,9


In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.33.159.18:8470
REPLICAS:  8


In [ ]:
# train_dataset = pd.read_excel('0818_cc_train_token_title.xlsx')
train_dataset = pd.read_excel('0818_cc_train_token_abstract2.xlsx')
train_dataset

,Unnamed: 0,Abstract,token,category,toxic
0,0,The urban heat island effect (UHI) is a produc...,urban heat-island UHU product urbanization cer...,green,21
1,1,Global warming is increasing extreme heat cond...,global extreme heat condition energy efficienc...,energy,11
2,2,Renewable energy is considered crucial for cli...,renewable energy crucial climate-change mitiga...,energy,11
3,3,Coastal groundwater-dependent ecosystems (GDEs...,coastal groundwater-dependent ecosystem gde we...,water,23
4,4,Reductions in carbon emissions have been a foc...,reduction carbon-emission focus power sector s...,energy,11
...,...,...,...,...,...
13188,13188,Electrochemical batteries are essential for re...,electrochemical battery essential renewable su...,energy,11
13189,13189,Many cities across the United States have turn...,many city united state energy disclosure bench...,energy,11
13190,13190,There is a compelling body of evidence calling...,body reduction amount carbon dioxide CO2 manki...,energy,11
13191,13191,This paper presents a multi-criteria optimizat...,present multi-criterion optimization power sys...,energy,11


In [ ]:
train = train_dataset[['token','toxic']]
train

,token,toxic
0,urban heat-island UHU product urbanization cer...,21
1,global extreme heat condition energy efficienc...,11
2,renewable energy crucial climate-change mitiga...,11
3,coastal groundwater-dependent ecosystem gde we...,23
4,reduction carbon-emission focus power sector s...,11
...,...,...
13188,electrochemical battery essential renewable su...,11
13189,many city united state energy disclosure bench...,11
13190,body reduction amount carbon dioxide CO2 manki...,11
13191,present multi-criterion optimization power sys...,11


In [ ]:
print(train['token'].apply(lambda x:len(str(x).split())).max())
max_len = (round(train['token'].apply(lambda x:len(str(x).split())).max()/100)*100)+50
print(max_len)

1034
1050


In [ ]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''

    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [ ]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.token.values, train.toxic.values,
                                                  stratify=train.toxic.values,
                                                  random_state=42,
                                                  test_size=0.2, shuffle=True)

In [ ]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
# max_len = 1000

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
# Calculating the Vocabulary Size and the number of Answers

print("Vocabulary size={}".format(len(token.word_index)))
print("Number of Answers={}".format(token.document_count))

Vocabulary size=27678
Number of Answers=13193


In [ ]:
le = LabelEncoder()
num_classes=10

# 0          energy
# 1        industry
# 2           waste
# 3         traffic
# 4        building
# 5        land-use
# 6           green
# 7    biodiversity
# 8           water
# 9     heat island

ytrain = le.fit_transform(ytrain)
yvalid = le.transform(yvalid)

ytrain = to_categorical(ytrain)
yvalid = to_categorical(yvalid)

In [ ]:
ytrain

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
ytrain.shape

(10554, 10)

# Word Embeddings

While building our simple RNN models we talked about using word-embeddings , So what is word-embeddings and how do we get word-embeddings?
Here is the answer :
* https://www.coursera.org/learn/nlp-sequence-models/lecture/6Oq70/word-representation
* https://machinelearningmastery.com/what-are-word-embeddings/
<br> <br>
The latest approach to getting word Embeddings is using pretained GLoVe or using Fasttext. Without going into too much details, I would explain how to create sentence vectors and how can we use them to create a machine learning model on top of it and since I am a fan of GloVe vectors, word2vec and fasttext. In this Notebook, I'll be using the GloVe vectors. You can download the GloVe vectors from here http://www-nlp.stanford.edu/data/glove.840B.300d.zip or you can search for GloVe in datasets on Kaggle and add the file

In [ ]:
# load the GloVe vectors in a dictionary:

embeddings_index = {}
f = open('glove.6B.300d.txt','r',encoding='utf-8')
for line in tqdm(f):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

400001it [00:59, 6702.49it/s]

Found 400001 word vectors.


# LSTM's

## Basic Overview

Simple RNN's were certainly better than classical ML algorithms and gave state of the art results, but it failed to capture long term dependencies that is present in sentences . So in 1998-99 LSTM's were introduced to counter to these drawbacks.

## In Depth Understanding

Why LSTM's?
* https://www.coursera.org/learn/nlp-sequence-models/lecture/PKMRR/vanishing-gradients-with-rnns
* https://www.analyticsvidhya.com/blog/2017/12/fundamentals-of-deep-learning-introduction-to-lstm/

What are LSTM's?
* https://www.coursera.org/learn/nlp-sequence-models/lecture/KXoay/long-short-term-memory-lstm
* https://distill.pub/2019/memorization-in-rnns/
* https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21

# Code Implementation

We have already tokenized and paded our text for input to LSTM's

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300)) # 300은 차원 수
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 27678/27678 [00:00<00:00, 131120.26it/s]


LSTM(n, input_dim = k, input_length = k)

n : 메모리 셀의 개수(기억용량, 출력형태를 결정함)

In [ ]:
# %%time
with strategy.scope():

    # A simple LSTM with glove embeddings and one dense layer
    model = Sequential()
    model.add(Embedding(len(word_index) + 1,
                     300,
                     weights=[embedding_matrix],
                     input_length=max_len,
                     trainable=False))

    model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1050, 300)         8303700   
                                                                 
 lstm_2 (LSTM)               (None, 128)               219648    
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 8,532,254
Trainable params: 228,554
Non-trainable params: 8,303,700
_________________________________________________________________


In [ ]:
model.fit(xtrain_pad, ytrain, validation_data = (xvalid_pad, yvalid), epochs=50, batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/50
21/21 [==============================] - 20s 522ms/step - loss: 1.5374 - accuracy: 0.5595 - val_loss: 1.1394 - val_accuracy: 0.6139
Epoch 2/50
21/21 [==============================] - 4s 210ms/step - loss: 0.8604 - accuracy: 0.7373 - val_loss: 0.6584 - val_accuracy: 0.7935
Epoch 3/50
21/21 [==============================] - 4s 196ms/step - loss: 0.6228 - accuracy: 0.8236 - val_loss: 0.5429 - val_accuracy: 0.8492
Epoch 4/50
21/21 [==============================] - 4s 198ms/step - loss: 0.4966 - accuracy: 0.8586 - val_loss: 0.4586 - val_accuracy: 0.8598
Epoch 5/50
21/21 [==============================] - 4s 210ms/step - loss: 0.4158 - accuracy: 0.8785 - val_loss: 0.3967 - val_accuracy: 0.8810
Epoch 6/50
21/21 [==============================] - 4s 201ms/step - loss: 0.3472 - accuracy: 0.8967 - val_loss: 0.3575 - val_accuracy: 0.8871
Epoch 7/50
21/21 [==============================] - 4s 202ms/step - loss: 0.2982 - accuracy: 0.9063 - val_loss: 0.3028 - val_accuracy: 0.8950
Epoch

In [ ]:
scores = model.evaluate(xvalid_pad, yvalid)
print("Accuracy: %.2f%%" % (scores[1]*100))

83/83 [==============================] - 6s 51ms/step - loss: 0.2451 - accuracy: 0.9447
Accuracy: 94.47%


In [ ]:
model.save('10_LSTM_abs.h5', save_format='h5')
model.save('10_LSTM_abs.keras')

In [ ]:
import pickle
pickle.dump(model, open('10_LSTM_abs.pkl', 'wb'))

In [ ]:
yvalid_model = model.predict(xvalid_pad)
yvalid_model

83/83 [==============================] - 6s 52ms/step


array([[9.99994278e-01, 4.33920235e-07, 2.97350283e-07, ...,
        1.24250377e-07, 3.39081055e-07, 1.36664551e-06],
       [1.68569159e-06, 8.71564907e-07, 4.08738697e-06, ...,
        5.33344924e-08, 9.59572617e-07, 1.88099239e-06],
       [9.99980450e-01, 9.42607812e-07, 3.54496228e-06, ...,
        5.59505281e-07, 2.46163893e-07, 5.29590170e-06],
       ...,
       [9.99998689e-01, 2.81671674e-07, 9.33330924e-08, ...,
        3.60918371e-08, 3.26637377e-07, 1.70747398e-07],
       [4.63717261e-05, 4.80558128e-06, 2.75027855e-06, ...,
        1.53377505e-06, 5.42490497e-08, 1.04080675e-06],
       [3.58628808e-04, 1.54771507e-01, 5.29252793e-05, ...,
        4.93972038e-06, 1.67636415e-06, 1.28871425e-05]], dtype=float32)

0	energy
1	industry
2	waste
3	traffic
4	building
5	land-use
6	green
7	biodiversity
8	water
9	heat island

In [ ]:
print(xvalid[3])
print(np.where(yvalid_model[3] == np.max(yvalid_model[3])))
print(np.where(yvalid[3] == np.max(yvalid[3])))

print(xvalid[37])
print(np.where(yvalid_model[37] == np.max(yvalid_model[37])))
print(np.where(yvalid[37] == np.max(yvalid[37])))

various definition energy net-zero energy building case definition energy operation aspect energy use relate construction delivery building component hand concept net energy field ecological economics account energy production process commodity field renewable energy assessment concept net energy introduce environment methodology embody energy building component energy use operation definition life cycle zero energy LC-ZEB use net energy ratio NER aid building design life cycle perspective
(array([0]),)
(array([0]),)
potential climate-change mitigation measure building science literature sparse study efficiency adaptation strategy aim contribute topic extent design typical zero energy buildle nzeb house wallonia belgium current operation summer thermal comfort climate calibrated whole building energy simulation integration future climate data derive high-resolution climate model study potential evolution risk living room main bedroom house compliance criterion study passive strategy ho

In [ ]:
digital = pd.read_excel('0818_digital_train_token_abstract2.xlsx')
digital

,Unnamed: 0,Abstract,token
0,0,Technological advancements such as information...,technological advancement information communic...
1,1,It has been many years now the concept of smar...,many year smart city new era urban innovation ...
2,2,The Internet of Things (IoT) has become a hot ...,internet thing iot hot topic present tech-driv...
3,3,Cities are increasingly turning towards specia...,city specialize technology address issue relat...
4,4,With the rapid development of artificial intel...,rapid development artificial intelligence AI A...
...,...,...,...
16782,16782,Hazardous waste management (HWM) involves tran...,hazardou waste management HWM involve transpor...
16783,16783,"ResumoDesde os anos 80, centenas de muculmanos...",resumodesde centena afro-americano predominant...
16784,16784,IMPORTANCE Cardiac telerehabilitation (CTR) ha...,IMPORTANCE cardiac telerehabilitation CTR safe...
16785,16785,This article re-examines the 'neolithic revolu...,article re-examine neolithic revolution gordon...


In [ ]:
print(digital['token'].apply(lambda x:len(str(x).split())).max())
max_len = (round(digital['token'].apply(lambda x:len(str(x).split())).max()/100)*100)+50
print(max_len)

538
550


In [ ]:
xtest = digital.token.values

# using keras tokenizer here
token = text.Tokenizer(num_words=None)
# max_len = 1000

token.fit_on_texts(list(xtest))
xtest_seq = token.texts_to_sequences(xtest)

#zero pad the sequences
xtest_pad = pad_sequences(xtest_seq, maxlen=max_len)

word_index = token.word_index

In [ ]:
ytest_model = model.predict(xtest_pad)
ytest_model

525/525 [==============================] - 16s 26ms/step


array([[1.2553282e-04, 9.0840931e-06, 2.0484504e-04, ..., 9.1340644e-06,
        1.3825513e-01, 9.7461650e-04],
       [2.1237076e-06, 2.3603185e-07, 1.6511132e-07, ..., 1.4606187e-08,
        7.0721214e-08, 3.2310419e-07],
       [9.9999607e-01, 2.8593632e-07, 2.3232472e-07, ..., 2.0092499e-07,
        2.9967478e-07, 1.3182964e-06],
       ...,
       [4.2902302e-02, 6.0121086e-04, 8.4391050e-04, ..., 8.2514323e-03,
        1.0463865e-01, 2.9434912e-02],
       [1.2172610e-03, 1.7358665e-04, 3.2826651e-02, ..., 3.1536132e-02,
        7.5436473e-02, 3.8228978e-03],
       [9.9950528e-01, 5.9587333e-06, 2.4117791e-05, ..., 2.0567256e-06,
        1.2319607e-06, 6.9934831e-05]], dtype=float32)

In [ ]:
print(xtest[0])
print(np.where(ytest_model[0] == np.max(ytest_model[0]))[0].item())

technological advancement information communication technology ict artificial intelligence AI internet-of-thing iot popularity smart city smart movement last couple decade development smart home domain number smart home relate systematic evolution domain study help understanding historical vicissitude state-of-the-art trend smart home cluster study scientometric method analyse publish last decade scientometric finding reveal smart home literature steady growth last decade smart home ICT home automation home information management AI home automation domestic energy management iot home automation home-based healthcare area iot popular technology realise smart home limit urban perspective social issue home technology smart home strong smart city agenda
3


In [ ]:
word

,cate,toxic,class
0,energy,11,0
1,industry,12,1
2,waste,13,2
3,traffic,14,3
4,building,15,4
5,land-use,16,5
6,green,21,6
7,biodiversity,22,7
8,water,23,8
9,heat island,24,9


In [ ]:
predict = {'token': [], 'class': [], 'cate': []}
for i in tqdm(range(len(xtest))):
  predict['token'].append(xtest[i])
  predict['class'].append(np.where(ytest_model[i] == np.max(ytest_model[i]))[0].item())
  predict['cate'].append(word[(word['class'] == np.where(ytest_model[i] == np.max(ytest_model[i]))[0].item())]['cate'].values[0])
predict = pd.DataFrame(predict)
predict

100%|██████████| 16787/16787 [00:08<00:00, 1886.75it/s]


,token,class,cate
0,technological advancement information communic...,3,traffic
1,many year smart city new era urban innovation ...,3,traffic
2,internet thing iot hot topic present tech-driv...,0,energy
3,city specialize technology address issue relat...,3,traffic
4,rapid development artificial intelligence AI A...,0,energy
...,...,...,...
16782,hazardou waste management HWM involve transpor...,0,energy
16783,resumodesde centena afro-americano predominant...,8,water
16784,IMPORTANCE cardiac telerehabilitation CTR safe...,3,traffic
16785,article re-examine neolithic revolution gordon...,3,traffic


In [ ]:
pd.DataFrame(predict).to_excel('0818_digital_predict.xlsx', index=False)